# Task 4: Sequential Pattern Mining

In [130]:
import pandas as pd
import copy

## Loading the new dataset

In [177]:
df = pd.read_csv('../dataset/new_customer_supermarket.csv', sep='\t', index_col=0)
df

,BasketID,BasketDate,Sale,CustomerID,ProdID,ProdDescr,Qta,TotSale
0,539993,2011-04-01 10:00:00,1.95,13313.0,22386,JUMBO BAG PINK POLKADOT,10,19.50
1,539993,2011-04-01 10:00:00,0.42,13313.0,21499,BLUE POLKADOT WRAP,25,10.50
2,539993,2011-04-01 10:00:00,0.42,13313.0,21498,RED RETROSPOT WRAP,25,10.50
3,539993,2011-04-01 10:00:00,2.10,13313.0,22379,RECYCLING BAG RETROSPOT,5,10.50
4,539993,2011-04-01 10:00:00,1.25,13313.0,20718,RED RETROSPOT SHOPPER BAG,10,12.50
...,...,...,...,...,...,...,...,...
363572,581587,2011-09-12 12:50:00,0.85,12680.0,22613,PACK OF SPACEBOY NAPKINS,12,10.20
363573,581587,2011-09-12 12:50:00,2.10,12680.0,22899,CHILDRENS APRON DOLLY GIRL,6,12.60
363574,581587,2011-09-12 12:50:00,4.15,12680.0,23254,CHILDRENS CUTLERY DOLLY GIRL,4,16.60
363575,581587,2011-09-12 12:50:00,4.15,12680.0,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,16.60


In [178]:
df.dtypes

BasketID        int64
BasketDate     object
Sale          float64
CustomerID    float64
ProdID         object
ProdDescr      object
Qta             int64
TotSale       float64
dtype: object

In [179]:
df = df.astype({'BasketDate': 'datetime64',
                'BasketID': 'object',
                'CustomerID': 'object'})

In [180]:
df['ProdDescr'].nunique()

3678

In [181]:
import webcolors
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
colors = webcolors.CSS3_NAMES_TO_HEX.keys()

def lemmatize_descr(descr):
    return ' '.join(lemmatizer.lemmatize(token) for token in word_tokenize(descr.lower()) 
                    if not token in stop_words and 
                    not any(color in token for color in colors) and 
                    len(token) >= 3)

In [182]:
df['ProdDescr'] = df['ProdDescr'].apply(lemmatize_descr)
df

,BasketID,BasketDate,Sale,CustomerID,ProdID,ProdDescr,Qta,TotSale
0,539993,2011-04-01 10:00:00,1.95,13313,22386,jumbo bag polkadot,10,19.50
1,539993,2011-04-01 10:00:00,0.42,13313,21499,polkadot wrap,25,10.50
2,539993,2011-04-01 10:00:00,0.42,13313,21498,retrospot wrap,25,10.50
3,539993,2011-04-01 10:00:00,2.10,13313,22379,recycling bag retrospot,5,10.50
4,539993,2011-04-01 10:00:00,1.25,13313,20718,retrospot shopper bag,10,12.50
...,...,...,...,...,...,...,...,...
363572,581587,2011-09-12 12:50:00,0.85,12680,22613,pack spaceboy napkin,12,10.20
363573,581587,2011-09-12 12:50:00,2.10,12680,22899,childrens apron dolly girl,6,12.60
363574,581587,2011-09-12 12:50:00,4.15,12680,23254,childrens cutlery dolly girl,4,16.60
363575,581587,2011-09-12 12:50:00,4.15,12680,23255,childrens cutlery circus parade,4,16.60


In [183]:
df['ProdDescr'].nunique()

3259

## AprioriAll

In [230]:
# An event is a list of strings.
# A sequence is a list of events.
# A dataset is a list of sequences.
# Thus, a dataset is a list of lists of lists of strings.

dataset = [
    [['a'], ['a', 'b', 'c'], ['a', 'c'], ['c']],
    [['a'], ['c'], ['b', 'c']],
    [['a', 'b'], ['d'], ['c'], ['b'], ['c']],
    [['a'], ['c'], ['b'], ['c']]
]

In [231]:
"""
This is a simple recursive method that checks if subsequence is a subSequence of mainSequence
"""
def isSubsequence(mainSequence, subSequence):
    subSequenceClone = list(subSequence) # clone the sequence, because we will alter it
    return isSubsequenceRecursive(mainSequence, subSequenceClone) # start recursion

"""
Function for the recursive call of isSubsequence, not intended for external calls
"""
def isSubsequenceRecursive(mainSequence, subSequenceClone, start=0):
    # check if empty: end of recursion, all itemsets have been found
    if not subSequenceClone:
        return True
    # retrieves element of the subsequence and removes is from subsequence 
    firstElem = set(subSequenceClone.pop(0))
    # search for the first itemset...
    for i in range(start, len(mainSequence)):
        if set(mainSequence[i]).issuperset(firstElem):
            # and recurse
            return isSubsequenceRecursive(mainSequence, subSequenceClone, i + 1)
    return False

In [232]:
aSequence = [['a'], ['b', 'c'], ['d'], ['a', 'e']]

In [233]:
isSubsequence(aSequence, [['a'], ['d'], ['e']])

True

In [234]:
isSubsequence(aSequence, [['a'], ['b', 'c'], ['e']])

True

In [235]:
isSubsequence(aSequence, [['a'], ['b', 'd']])

False

In [236]:
"""
Computes the length of the sequence (sum of the length of the contained itemsets)
"""
def sequenceLength(sequence):
    return sum(len(i) for i in sequence)

In [237]:
sequenceLength([['a'], ['b', 'c'], ['a'], ['b', 'c', 'd']])

7


In [238]:
"""
Computes the support of a sequence in a dataset
"""
def countSupport(dataset, candidateSequence):
    return sum(1 for seq in dataset if isSubsequence(seq, candidateSequence))

In [239]:
countSupport(dataset, [['b']])

4

In [240]:
countSupport(dataset, [['a'], ['b', 'c']])

2

In [241]:
"""
Generates one candidate of length k from two candidates of length (k-1) as used in the AprioriAll algorithm
"""
def generateCandidatesForPair(cand1, cand2):
    cand1Clone = copy.deepcopy(cand1)
    cand2Clone = copy.deepcopy(cand2)
    # drop the leftmost item from cand1:
    if len(cand1[0]) == 1:
        cand1Clone.pop(0)
    else:
        cand1Clone[0] = cand1Clone[0][1:]
    # drop the rightmost item from cand2:
    if len(cand2[-1]) == 1:
        cand2Clone.pop(-1)
    else:
        cand2Clone[-1] = cand2Clone[-1][:-1]
    
    # if the result is not the same, then we dont need to join
    if not cand1Clone == cand2Clone:
        return []
    else:
        newCandidate = copy.deepcopy(cand1)
        if len(cand2[-1]) == 1:
            newCandidate.append(cand2[-1])
        else:
            newCandidate[-1].extend(cand2[-1][-1])
        return newCandidate

In [242]:
candidateA = [['a'], ['b', 'c'], ['d']]
candidateB = [['b', 'c'], ['d', 'e']]
generateCandidatesForPair(candidateA, candidateB)

[['a'], ['b', 'c'], ['d', 'e']]

In [243]:
candidateA = [['a'], ['b', 'c'], ['d']]
candidateC = [['b', 'c'], ['d'], ['e']]
generateCandidatesForPair(candidateA, candidateC)

[['a'], ['b', 'c'], ['d'], ['e']]

In [244]:
candidateA = [['a'], ['b', 'c'], ['d']]
candidateD = [['a'], ['b', 'c'], ['e']]
generateCandidatesForPair(candidateA, candidateD)

[]

In [245]:
"""
Generates the set of candidates of length k from the set of frequent sequences with length (k-1)
"""
def generateCandidates(lastLevelCandidates):
    k = sequenceLength(lastLevelCandidates[0]) + 1
    if k == 2:
        flatShortCandidates = [item for sublist2 in lastLevelCandidates for sublist1 in sublist2 for item in sublist1]
        result = [[[a, b]] for a in flatShortCandidates for b in flatShortCandidates if b > a]
        result.extend([[[a], [b]] for a in flatShortCandidates for b in flatShortCandidates])
        return result
    else:
        candidates = []
        for i in range(0, len(lastLevelCandidates)):
            for j in range(0, len(lastLevelCandidates)):
                newCand = generateCandidatesForPair(lastLevelCandidates[i], lastLevelCandidates[j])
                if not newCand == []:
                    candidates.append(newCand)
        candidates.sort()
        return candidates

An example; lets assume, we know the frequent sequences of level 2:

In [246]:
lastLevelFrequentPatterns = [
    [['a', 'b']], 
    [['b', 'c']], 
    [['a'], ['b']], 
    [['a'], ['c']], 
    [['b'], ['c']], 
    [['c'], ['b']], 
    [['c'], ['c']]
]

Then we can compute the generate candidates for level 3

In [247]:
newCandidates = generateCandidates(lastLevelFrequentPatterns)
newCandidates

[[['a'], ['b'], ['c']],
 [['a'], ['b', 'c']],
 [['a'], ['c'], ['b']],
 [['a'], ['c'], ['c']],
 [['a', 'b'], ['c']],
 [['a', 'b', 'c']],
 [['b'], ['c'], ['b']],
 [['b'], ['c'], ['c']],
 [['b', 'c'], ['b']],
 [['b', 'c'], ['c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['c'], ['b']],
 [['c'], ['c'], ['c']]]

In [248]:
"""
Computes all direct subsequence for a given sequence.
A direct subsequence is any sequence that originates from deleting exactly one item from any event in the original sequence.
"""
def generateDirectSubsequences(sequence):
    result = []
    for i, itemset in enumerate(sequence):
        if len(itemset) == 1:
            sequenceClone = copy.deepcopy(sequence)
            sequenceClone.pop(i)
            result.append(sequenceClone)
        else:
            for j in range(len(itemset)):
                sequenceClone = copy.deepcopy(sequence)
                sequenceClone[i].pop(j)
                result.append(sequenceClone)
    return result

In [249]:
"""
Prunes the set of candidates generated for length k given all frequent sequence of level (k-1), as done in AprioriAll
"""
def pruneCandidates(candidatesLastLevel, candidatesGenerated):
    return [cand for cand in candidatesGenerated if all(x in candidatesLastLevel for x in generateDirectSubsequences(cand))]

We apply this on example dataset:

In [250]:
candidatesPruned = pruneCandidates(lastLevelFrequentPatterns, newCandidates)
candidatesPruned

[[['a'], ['b'], ['c']],
 [['a'], ['b', 'c']],
 [['a'], ['c'], ['b']],
 [['a'], ['c'], ['c']],
 [['a', 'b'], ['c']],
 [['b'], ['c'], ['c']],
 [['b', 'c'], ['c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['c'], ['b']],
 [['c'], ['c'], ['c']]]

In [251]:
minSupport = 2
candidatesCounts = [(i, countSupport(dataset, i)) for i in candidatesPruned]
resultLvl = [(i, count) for i, count in candidatesCounts if count >= minSupport]
resultLvl

[([['a'], ['b'], ['c']], 3),
 ([['a'], ['b', 'c']], 2),
 ([['a'], ['c'], ['b']], 3),
 ([['a'], ['c'], ['c']], 4),
 ([['a', 'b'], ['c']], 2),
 ([['b'], ['c'], ['c']], 2),
 ([['c'], ['b'], ['c']], 2)]

In [252]:
"""
The AprioriAll algorithm with time constraints. Computes the frequent sequences in a seqeunce dataset.

Args:
    dataset: a list of sequences, for which the frequent (sub-)sequences are computed
    minSupport: the minimum support that makes a sequence frequent
    max_span: this constraint specifies the maximum allowed time difference between the latest 
              and the earliest occurrences of events in the entire sequence
    min_gap: this constraint specifies the minimum allowed time difference between the latest 
              and the earliest element of the pattern instance
    max_gap: this constraint specifies the maximum allowed time difference between the latest 
              and the earliest element of the pattern instance
    verbose: if true, additional information on the mining process is printed (i.e., candidates on each level)

Returns:
    A list of tuples (s, c), where s is a frequent sequence and c is the count for that sequence
"""
def aprioriall(dataset, minSupport, max_span, min_gap=1, max_gap=np.inf, verbose=False):
    global numberOfCountingOperations
    numberOfCountingOperations = 0
    overall = []
    itemsInDataset = sorted(set([item for sublist1 in dataset for sublist2 in sublist1 for item in sublist2]))
    singleItemSequences = [[[item]] for item in itemsInDataset]
    singleItemCounts = [(i, countSupport(dataset, i)) for i in singleItemSequences if countSupport(dataset, i) >= minSupport]
    overall.append(singleItemCounts)
    print('Result, lvl 1: ' + str(overall[0]))
    k = 1
    while True:
        if not overall[k - 1]:
            break
        # 1. candidate generation
        candidatesLastLevel = [x[0] for x in overall[k - 1]]
        candidatesGenerated = generateCandidates(candidatesLastLevel)
        # 2. candidate pruning (using a "containsall" subsequences)
        candidatesPruned = [cand for cand in candidatesGenerated if all(x in candidatesLastLevel for x in generateDirectSubsequences(cand))]
        # 3. candidate checking
        candidatesCounts = [(i, countSupport(dataset, i)) for i in candidatesPruned]
        resultLvl = [(i, count) for i, count in candidatesCounts if count >= minSupport]
        if verbose:
            print('Candidates generated, lvl ' + str(k + 1) + ': ' + str(candidatesGenerated))
            print('Candidates pruned, lvl ' + str(k + 1) + ': ' + str(candidatesPruned))
            print('Result, lvl ' + str(k + 1) + ': ' + str(resultLvl))
        overall.append(resultLvl)
        k = k + 1
    # "flatten" overall
    overall = overall[:-1]
    overall = [item for sublist in overall for item in sublist]
    return overall

In [253]:
aprioriall(dataset, minSupport=2, verbose=False)

Result, lvl 1: [([['a']], 4), ([['b']], 4), ([['c']], 4)]


[([['a']], 4),
 ([['b']], 4),
 ([['c']], 4),
 ([['a', 'b']], 2),
 ([['b', 'c']], 2),
 ([['a'], ['b']], 4),
 ([['a'], ['c']], 4),
 ([['b'], ['c']], 3),
 ([['c'], ['b']], 3),
 ([['c'], ['c']], 4),
 ([['a'], ['b'], ['c']], 3),
 ([['a'], ['b', 'c']], 2),
 ([['a'], ['c'], ['b']], 3),
 ([['a'], ['c'], ['c']], 4),
 ([['a', 'b'], ['c']], 2),
 ([['b'], ['c'], ['c']], 2),
 ([['c'], ['b'], ['c']], 2),
 ([['a'], ['c'], ['b'], ['c']], 2),
 ([['a', 'b'], ['c'], ['c']], 2)]

### Supermarket dataset

In [286]:
baskets_sequence = pd.DataFrame(df.groupby(['CustomerID', 'BasketID', 'BasketDate'])['ProdDescr'].apply(list)).sort_values('BasketDate')
baskets_sequence

ProdDescr
CustomerID BasketID BasketDate                                                            
15240.0    542776   2011-01-02 08:23:00  [spot ceramic drawer knob, stripe ceramic draw...
14911.0    542777   2011-01-02 08:31:00  [airline bag vintage jet set, airline bag vint...
14496.0    542778   2011-01-02 09:01:00  [strawberry raffia food cover, clear drawer kn...
17147.0    542779   2011-01-02 09:36:00  [set polkadot party candle, set polkadot party...
17675.0    542780   2011-01-02 09:38:00  [hot bath metal sign, lady gentleman metal sig...
...                                                                                    ...
16945.0    570864   2011-12-10 15:41:00  [feltcraft butterfly heart, christmas pudding ...
           570865   2011-12-10 15:42:00  [mini playing card gymkhana, mini playing card...
17096.0    570866   2011-12-10 15:49:00  [retrospot heart hot water bottle, scottie dog...
12667.0    570875   2011-12-10 17:00:00  [baking set spaceboy design, retrospot sugar j...
16085.0    570876   2011-12-10 17:19:00  [polyester filler pad, polyester filler pad, p...

[16950 rows x 1 columns]

In [287]:
from itertools import groupby

baskets_sequence['ProdDescr'] = baskets_sequence['ProdDescr'].apply(lambda prods: [prod[0] for prod in groupby(prods)]) # removing consecutive duplicates
baskets_sequence

ProdDescr
CustomerID BasketID BasketDate                                                            
15240.0    542776   2011-01-02 08:23:00  [spot ceramic drawer knob, stripe ceramic draw...
14911.0    542777   2011-01-02 08:31:00  [airline bag vintage jet set, airline bag vint...
14496.0    542778   2011-01-02 09:01:00  [strawberry raffia food cover, clear drawer kn...
17147.0    542779   2011-01-02 09:36:00  [set polkadot party candle, easter tin keepsak...
17675.0    542780   2011-01-02 09:38:00  [hot bath metal sign, lady gentleman metal sig...
...                                                                                    ...
16945.0    570864   2011-12-10 15:41:00  [feltcraft butterfly heart, christmas pudding ...
           570865   2011-12-10 15:42:00  [mini playing card gymkhana, mini playing card...
17096.0    570866   2011-12-10 15:49:00  [retrospot heart hot water bottle, scottie dog...
12667.0    570875   2011-12-10 17:00:00  [baking set spaceboy design, retrospot sugar j...
16085.0    570876   2011-12-10 17:19:00  [polyester filler pad, poste france cushion co...

[16950 rows x 1 columns]

In [288]:
baskets_sequence = pd.DataFrame(baskets_sequence.groupby('CustomerID')['ProdDescr'].apply(list))
baskets_sequence

,ProdDescr
CustomerID,
12347.0,"[[new baroquecandlestick candle, new baroque c..."
12348.0,"[[pack retrospot tissue, pack heart design tis..."
12349.0,"[[parisienne curio cabinet, sweetheart wall ti..."
12350.0,"[[way metal sign, metal sign neighbourhood wit..."
12352.0,"[[deluxe sewing kit, heart shape egg frying pa..."
...,...
18280.0,"[[wood board ant finish, retrospot lamp, gumba..."
18281.0,"[[robot birthday card, card circus parade, pen..."
18282.0,"[[regency tier, rose regency teacup saucer, ca..."


In [295]:
baskets_sequence['ProdDescr'].tolist()[:3]

[[['new baroquecandlestick candle',
   'new baroque candlestick candle',
   'candelabra tlight holder',
   'woodland charlotte bag',
   'airline bag vintage jet set',
   'sandwich bath sponge',
   'alarm clock bakelike',
   'small heart measuring spoon',
   'sweetheart fairy cake case',
   'teatime fairy cake case',
   'pack mushroom cake case',
   'pack spaceboy cake case',
   'tea time oven glove',
   'retrospot oven glove',
   'retrospot oven glove double',
   'set retrospot tea towel',
   'regency tier',
   'box assorted colour teaspoon',
   'mini ladle love heart',
   'calculator',
   'toothpaste tube pen',
   'set tin vintage bathroom',
   'toadstool led night light',
   'dog picture playing card'],
  ['set vintage leaf cake case',
   'set heart shape petit four case',
   'airline bag vintage jet set',
   'airline bag vintage tokyo',
   'airline bag vintage world champion',
   'woodland design cotton tote bag',
   'woodland charlotte bag',
   'alarm clock bakelike',
   'triple ho

In [289]:
aprioriall(baskets_sequence['ProdDescr'].tolist(), minSupport=400, verbose=False)

Result, lvl 1: [([['alarm clock bakelike']], 496), ([['assorted colour bird ornament']], 651), ([['baking set piece retrospot']], 576), ([['gardener kneeling pad keep calm']], 405), ([['hanging heart tlight holder']], 883), ([['harmonica box']], 426), ([['heart wicker large']], 440), ([['heart wicker small']], 529), ([['hot water bottle keep calm']], 473), ([['jam making set jar']], 530), ([['jam making set printed']], 477), ([['jumbo bag retrospot']], 609), ([['kitchen scale']], 470), ([['lunch bag apple design']], 400), ([['lunch bag car']], 466), ([['lunch bag polkadot']], 436), ([['lunch bag retrospot']], 518), ([['lunch bag skull']], 449), ([['lunch bag spaceboy design']], 476), ([['lunch bag suki design']], 423), ([['natural slate heart chalkboard']], 556), ([['pack retrospot cake case']], 602), ([['paper chain kit christmas']], 553), ([['paper chain kit vintage christmas']], 414), ([['party bunting']], 699), ([['rabbit night light']], 449), ([['recipe box pantry design']], 450),

[([['alarm clock bakelike']], 496),
 ([['assorted colour bird ornament']], 651),
 ([['baking set piece retrospot']], 576),
 ([['gardener kneeling pad keep calm']], 405),
 ([['hanging heart tlight holder']], 883),
 ([['harmonica box']], 426),
 ([['heart wicker large']], 440),
 ([['heart wicker small']], 529),
 ([['hot water bottle keep calm']], 473),
 ([['jam making set jar']], 530),
 ([['jam making set printed']], 477),
 ([['jumbo bag retrospot']], 609),
 ([['kitchen scale']], 470),
 ([['lunch bag apple design']], 400),
 ([['lunch bag car']], 466),
 ([['lunch bag polkadot']], 436),
 ([['lunch bag retrospot']], 518),
 ([['lunch bag skull']], 449),
 ([['lunch bag spaceboy design']], 476),
 ([['lunch bag suki design']], 423),
 ([['natural slate heart chalkboard']], 556),
 ([['pack retrospot cake case']], 602),
 ([['paper chain kit christmas']], 553),
 ([['paper chain kit vintage christmas']], 414),
 ([['party bunting']], 699),
 ([['rabbit night light']], 449),
 ([['recipe box pantry desig

In [296]:
import re
from collections import defaultdict
from functools import lru_cache
from itertools import product
from math import floor

class DataSequence(object):
    """Represents a data sequence."""
    def __init__(self, elements):
        self.elements = elements
        self.items = set(flatten(elements))

    def __repr__(self):
        return repr(self.elements)

class SequentialPattern(object):
    """Represents a sequential pattern."""

    def __init__(self, items, support):
        self.items = items
        self.support = support

    def __repr__(self):
        return "< {} > ({:.3g})".format(", ".join(self.items), self.support)

class ResultSet(object):
    """The results of a run of the apriori algorithm."""
    def __init__(self, frequent_sequences):
        self.frequent_sequences = frequent_sequences
        self.max_len = len(frequent_sequences)
        self.num_of_sequences = len(flatten(self.frequent_sequences))

    def all(self):
        return ResultSet.__sort(flatten(self.frequent_sequences))

    def __getitem__(self, i):
        return ResultSet.__sort(self.frequent_sequences[i])

    @staticmethod
    def __sort(sequences):
        return sorted(sequences, key=lambda x: x.support, reverse=True)

    def __repr__(self):
        s = ""
        for pattern in self.all():
            s += repr(pattern) + "\n"
        s += "Number of patterns {}\n".format(self.num_of_sequences)
        s += "Largest pattern is of size {}\n".format(self.max_len)
        return s


def read_data_sequences(filename):
    """Read data sequences from a file containing lines of the format:
        { item_id item_id }{ item_id }...{item_id item_id item_id} etc."""
    data_sequences = []
    for f in open(filename, "r"):
        elements = []
        for element in re.findall(r"{[\w\s]+}", f):
            elements.append(set(re.findall(r"\w+", element)))
        data_sequences.append(DataSequence(elements))
    return data_sequences

def apriori(data_sequences, minsup, maxgap=999, mingap=1):
    """Apriori-like algorithm for mining sequential patterns."""
    frequent_seqs = []
    n = len(data_sequences)
    min_support_count = floor(minsup * n)

    frequent_seqs.append(find_frequent_1_subsequences(data_sequences,
        min_support_count))

    k = 2
    while frequent_seqs[-1]:
        frequent_seqs.append([])
        candidates = apriori_gen(frequent_seqs[-2])

        for candidate in candidates:
            support_count = calculate_support(candidate, data_sequences,
                    maxgap, mingap)
            if support_count >= min_support_count:
                pattern = SequentialPattern(candidate, support_count / n)
                frequent_seqs[-1].append(pattern)
        print("... of which {} are {}-patterns.\n".format(len(frequent_seqs[-1]), k))
        k += 1

    return ResultSet(frequent_seqs)


def find_frequent_1_subsequences(data_sequences, min_support_count):
    """Find all frequent 1-subsequences."""
    support_counts = defaultdict(int)
    n = len(data_sequences)

    # Calculate support counts
    for data_sequence in data_sequences:
        for item in data_sequence.items:
            support_counts[item] += 1

    # Filter subsequences above minsup
    frequent_seqs = []
    for k,v in support_counts.items():
        if v >= min_support_count:
            frequent_seqs.append(SequentialPattern([k], v / n))
    print("Generated {} frequent 1-patterns.\n".format(len(frequent_seqs)))
    return frequent_seqs

def apriori_gen(prev_subsequences):
    """Generate candidate k-subsequences given frequent k-1 subsequences."""
    candidates = []
    prev_subsequences = map(lambda pattern: pattern.items, prev_subsequences)
    for seq1, seq2 in product(prev_subsequences, repeat=2):
        if seq1[1:] == seq2[:-1]:
            candidates.append(seq1 + [seq2[-1]])
    print("Generated {} candidates...".format(len(candidates)))
    return candidates

def calculate_support(candidate, data_sequences, maxgap, mingap):
    """Calculate a candidate's support given a list of data sequences."""
    support_count = 0
    for data_sequence in data_sequences:
        if occurs(candidate, data_sequence, maxgap, mingap):
            support_count += 1
    return support_count

def occurs(candidate, data_sequence, maxgap, mingap):
    """Check whether given candidate sequence occurs in given data sequence."""
    matchings = list_matchings(candidate, data_sequence)
    if matchings:
        return find_occurrence(candidate, matchings, maxgap, mingap)
    else:
        return False


def list_matchings(candidate, data_sequence):
    """Find all matchings of candidate events in data sequence."""

    for item in candidate:
        if item not in data_sequence.items:
            return False

    matchings = []
    last_min = 0
    for item in candidate:
        matching = []
        for j in range(last_min, len(data_sequence.elements)):
            if item in data_sequence.elements[j]:
                matching.append(j)
        if matching:
            matchings.append(matching)
            last_min = matching[0] + 1
        else:
            return False
    return matchings

def find_occurrence(candidate, matchings, maxgap, mingap):
    """Find an occurrence of a candidate pattern against given matchings, also
    satisfying maxgap and mingap constraints."""
    stack = []
    level = 0
    index = 0
    while len(stack) != len(candidate):
        if level == 0:
            if index == len(matchings[0]):
                return False
            stack.append(index)
            level += 1
            index = 0
        elif index == len(matchings[level]):
            level -= 1
            index = stack.pop() + 1
        elif matchings[level][index] > matchings[level-1][stack[-1]]:
            diff = matchings[level][index] - matchings[level-1][stack[-1]]
            if diff > maxgap: # Max gap
                level -= 1
                index = stack.pop() + 1
            elif diff < mingap: # Min gap
                index += 1
            else:
                stack.append(index)
                level += 1
                index = 0
        else:
            index += 1
    return stack

In [298]:
apriori(baskets_sequence['ProdDescr'].tolist(), minsup=10)

AttributeError: 'list' object has no attribute 'items'